In [3]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Conv2D,Dense,Flatten,MaxPooling2D,Dropout
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
# np.set_printoptions(threshold=sys.maxsize)

In [4]:
folders = [f'0{x}' for x in range(0,10)]
labels = []
file_names = []
for folder in folders:
    all_subfolders = [f for f in os.listdir(f'Images/{folder}') if not f.startswith('.')]
    # print(all_subfolders , folder)
    for subfolder in all_subfolders:
        all_entries = os.listdir(f'Images/{folder}/{subfolder}')
        names = [f'Images/{folder}/{subfolder}/{entry}' for entry in all_entries if os.path.isfile(os.path.join(f'Images/{folder}/{subfolder}', entry))]
        file_names.append(names)
        gesture = "".join(subfolder.split('_')[1:])
        labels.append(gesture)
    file_names
print(len(file_names))

100


In [6]:
label_list = ['palm','l','fist','fistmoved','thumb','index','ok','palmmoved','c','down']
label_encoder = LabelEncoder()
integer_labels = label_encoder.fit_transform(label_list)
print(integer_labels)
one_hot_labels = to_categorical(integer_labels)
dic = {}
i = 0
for label in integer_labels:
    dic[label_list[i]] = label
    i+=1
dic

[7 5 2 3 9 4 6 8 0 1]


{'palm': 7,
 'l': 5,
 'fist': 2,
 'fistmoved': 3,
 'thumb': 9,
 'index': 4,
 'ok': 6,
 'palmmoved': 8,
 'c': 0,
 'down': 1}

In [7]:
len(file_names)

100

In [8]:
images = []
gestures = []
i = 0
for file_name in file_names:
    for name in file_name:
        image = cv2.imread(name)
        images.append(image)
        gestures.append(dic[labels[i]])
    i+=1
        
        

In [38]:
resized_images = []
for img in images:
    resized_img = cv2.resize(img, (196, 196))
    resized_images.append(resized_img)

resized_images = np.array(resized_images)

In [22]:
gestures = np.reshape(gestures,(20000,1))

In [15]:
gestures[2000]

array([9])

In [16]:
model = Sequential()
model.add(Conv2D(96,(3,3),activation='relu',input_shape = (196,196,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(196,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Flatten())
model.add(Dense(512))
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

learning_rate_reduction = ReduceLROnPlateau(monitor="loss", 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)


/opt/anaconda3/envs/tf-metal/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-25 14:38:23.669025: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-25 14:38:23.681171: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-25 14:38:23.681186: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-25 14:38:23.681221: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-25 14:38:23.681245: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 194, 194, 96)   │         2,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 97, 97, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 95, 95, 128)    │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 47, 47, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 45, 45, 196)    │       225,988 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 22, 22, 196)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 20, 20, 256)    │       451,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,644,494 (13.90 MB)

 Trainable params: 3,644,494 (13.90 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
len(gestures)

20000

In [23]:
gestures_onehot = to_categorical(gestures, num_classes=10)

In [39]:
# model.fit(resized_images,gestures_onehot,epochs=5,batch_size=32,verbose=2,callbacks=learning_rate_reduction)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(resized_images, gestures_onehot, test_size=0.2, random_state=42)
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_val, y_val), verbose=2, callbacks=[learning_rate_reduction])

Epoch 1/5
125/125 - 242s - 2s/step - accuracy: 0.7311 - loss: 3.4019 - val_accuracy: 0.9740 - val_loss: 0.1160 - learning_rate: 5.0000e-04
Epoch 2/5
125/125 - 230s - 2s/step - accuracy: 0.9808 - loss: 0.0679 - val_accuracy: 0.9910 - val_loss: 0.0331 - learning_rate: 5.0000e-04
Epoch 3/5
125/125 - 244s - 2s/step - accuracy: 0.9931 - loss: 0.0262 - val_accuracy: 0.9945 - val_loss: 0.0178 - learning_rate: 5.0000e-04
Epoch 4/5
125/125 - 233s - 2s/step - accuracy: 0.9954 - loss: 0.0159 - val_accuracy: 0.9948 - val_loss: 0.0136 - learning_rate: 5.0000e-04
Epoch 5/5
125/125 - 245s - 2s/step - accuracy: 0.9929 - loss: 0.0224 - val_accuracy: 0.9925 - val_loss: 0.0267 - learning_rate: 5.0000e-04


In [41]:
model.save('handRecogniser1.h5')

In [46]:
testing_images = resized_images[19000:]
testing_outputs = gestures[19000:]
pred = model.predict(testing_images)

32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step 


In [54]:
testing_outputs.shape

(1000, 1)

In [56]:
df = pd.DataFrame({'predicted':pred,"actual":testing_outputs.reshape(1000)
})

In [57]:
df

,predicted,actual
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
995,3,3
996,3,3
997,3,3
998,3,3


In [59]:
df[df['predicted']!=df['actual']]

,predicted,actual
382,8,4
